<a href="https://colab.research.google.com/github/swetha028-git/Isolation/blob/main/ctsp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install imbalanced-learn

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

# Mount your Google Drive to Colab
from google.colab import drive
drive.mount('/content/gdrive')



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
train_dir = '/content/gdrive/MyDrive/blood_cancer/train/train'
test_dir = '/content/gdrive/MyDrive/blood_cancer/test/test'
val_dir = '/content/gdrive/MyDrive/blood_cancer/val/val'

In [ ]:

def contrast_augmentation(image):
    import numpy as np
    import cv2
    contrast_factor = np.random.uniform(0.5, 1.5)
    image = cv2.convertScaleAbs(image, alpha=contrast_factor, beta=0)
    return image

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    featurewise_center=True,
    featurewise_std_normalization=True
)

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)



In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 4961 images belonging to 2 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 1249 images belonging to 2 classes.


In [ ]:
base_model = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=base_model.input, outputs=x)

# Compile model with different hyperparameters
# Compile model with different hyperparameters
model.compile(optimizer=keras.optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall', 'auc'])

# Define early stopping callback with patience



In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights = dict(enumerate(class_weights))

# Define early stopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)


In [ ]:
class EvaluateModel(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        y_pred = model.predict(test_generator)
        y_pred_class = np.argmax(y_pred, axis=1)
        conf_mat = confusion_matrix(test_generator.classes, y_pred_class)
        print(f'Epoch {epoch+1}, Confusion Matrix:\n{conf_mat}')


In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=20,
    class_weight=class_weights,
    callbacks=[early_stopping, EvaluateModel()]
)

Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 155ms/step
Epoch 1, Confusion Matrix:
[[629   0]
 [620   0]]
156/156 ━━━━━━━━━━━━━━━━━━━━ 133s 752ms/step - accuracy: 0.6316 - auc: 0.6854 - loss: 0.6869 - precision: 0.6321 - recall: 0.6316 - val_accuracy: 0.9912 - val_auc: 0.9998 - val_loss: 0.3431 - val_precision: 0.9967 - val_recall: 0.9855
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step
Epoch 2, Confusion Matrix:
[[629   0]
 [620   0]]
156/156 ━━━━━━━━━━━━━━━━━━━━ 132s 724ms/step - accuracy: 0.9368 - auc: 0.9834 - loss: 0.1713 - precision: 0.9308 - recall: 0.9411 - val_accuracy: 0.9912 - val_auc: 1.0000 - val_loss: 0.0847 - val_precision: 1.0000 - val_recall: 0.9823
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 132ms/step
Epoch 3, Confusion Matrix:
[[629   0]
 [620   0]]
156/156 ━━━━━━━━━━━━━━━━━━━━ 139s 699ms/step - accuracy: 0.9735 - auc: 0.9950 - loss: 0.0918 - precision: 0.9758 - recall: 0.9698 - val_accuracy: 0.9896 - val_auc: 1.0000 - val_loss: 0.0445 - val_precision: 1.0000 - val_reca

In [ ]:
loss, accuracy, precision, recall, auc = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy:.2f}')
print(f'Test precision: {precision:.2f}')
print(f'Test recall: {recall:.2f}')
print(f'Test AUC: {auc:.2f}')

40/40 ━━━━━━━━━━━━━━━━━━━━ 7s 160ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 0.0032 - precision: 1.0000 - recall: 1.0000
Test accuracy: 1.00
Test precision: 1.00
Test recall: 1.00
Test AUC: 1.00


In [ ]:
def predict_image(image_path):
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    prediction = model.predict(img_array)
    print(f'Prediction: {prediction[0][0]:.2f}')
    if prediction[0][0] > 0.5:
        print('Class: 1')
    else:
        print('Class: 0')

# Test the function
image_path = '/content/gdrive/MyDrive/blood_cancer/val/val/Cancer/c1.jpeg'
predict_image(image_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Prediction: 0.00
Class: 0
